#Trần Hoàn Đức Duy - 19521434

In [375]:
# %cd /content/drive/MyDrive/Programming/Python for machine learning

#Bước 1: đọc dữ liệu từ file excel/*csv*

In [376]:
import pandas as pd

In [377]:
ds = pd.read_csv('data/Churn_Modelling.csv')
ds.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [378]:
x,y = ds.iloc[:,3:-1], ds.iloc[:,-1]

In [379]:
x.tail()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52
9999,792,France,Female,28,4,130142.79,1,1,0,38190.78


#Bước 2: chuẩn hóa các cột dữ liệu thuộc dạng không có tình thứ tự (dữ liệu rời rạc) thành dạng One Hot Encoding

In [380]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder, Normalizer
from sklearn.compose import ColumnTransformer
import numpy as np

In [381]:
le_1 = LabelEncoder()
x.iloc[:,1] = le_2.fit_transform(x.iloc[:,1])
le_2 = LabelEncoder()
x.iloc[:,2] = le_2.fit_transform(x.iloc[:,2])

In [382]:
columnTransfer = ColumnTransformer([('encoder', OneHotEncoder(),[1,2])],remainder = 'passthrough')
x = np.array(columnTransfer.fit_transform(x))

In [383]:
x[0]

array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       0.0000000e+00, 6.1900000e+02, 4.2000000e+01, 2.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0134888e+05])

#Bước 3: Chia dữ liệu ra làm 2 tập train và test 

In [384]:
from sklearn.model_selection import train_test_split

In [385]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2)

#Bước 4: Chuẩn hóa dữ liệu train bằng standard scaler

In [386]:
from sklearn.preprocessing import StandardScaler

In [387]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

#Bước 5: Xây dựng mô hình Mạng Neural Network bằng thư viện Keras và lớp đối tượng Dense

In [388]:
import torch
from torch import nn

In [389]:
num_features = x_train.shape[1]
num_output = 1
num_features,num_output

(13, 1)

In [390]:
net = nn.Sequential(nn.Linear(num_features,6),nn.ReLU(),nn.Linear(6,1),nn.Sigmoid())

In [391]:
net.eval()

Sequential(
  (0): Linear(in_features=13, out_features=6, bias=True)
  (1): ReLU()
  (2): Linear(in_features=6, out_features=1, bias=True)
  (3): Sigmoid()
)

#Bước 6: huấn luyện mạng với thuật toán Adam sử dụng hàm độ lỗi Binary Cross Entropy

In [392]:
x_train=torch.from_numpy(x_train.astype(np.float32))
x_test=torch.from_numpy(x_test.astype(np.float32))
y_train=torch.from_numpy(y_train.to_numpy().reshape(-1,1).astype(np.float32))
y_test=torch.from_numpy(y_test.to_numpy().reshape(-1,1).astype(np.float32))

In [393]:
num_iters = 10000

In [394]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(),lr=0.3)

In [395]:
for iter in range(num_iters):
  optimizer.zero_grad()
  outputs = net(x_train)
  loss = criterion(outputs,y_train)
  loss.backward()
  optimizer.step()

  if iter%1000 == 0:
    print('loss: ', loss.item())

loss:  0.6502276659011841
loss:  0.3303644359111786
loss:  0.3298073410987854
loss:  0.3309177756309509
loss:  0.3301394581794739
loss:  0.330552875995636
loss:  0.3307734727859497
loss:  0.3297865688800812
loss:  0.32976245880126953
loss:  0.33129358291625977


#Bước 7: test và đánh giá kết quả

In [396]:
with torch.no_grad():
  y_train_pred = net(x_train).round()
  y_test_pred = net(x_test).round()
print('Train accuracy: ', ((y_train_pred == y_train).sum() / len(y_train)).numpy())
print('Test accuracy: ', ((y_test_pred == y_test).sum() / len(y_test)).numpy())

Train accuracy:  0.864
Test accuracy:  0.8595


#Bước 8: đánh giá độ chính xác bằng phương pháp KFold Cross Validation

In [397]:
from sklearn.model_selection import KFold
def evaluation_kfold(X,y):
  kf = KFold(n_splits=5)
  train_accuracies = []
  val_accuracies = []
  for train_index, val_index in kf.split(X):

    x_train, x_test = X[train_index], X[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    scaler = StandardScaler()
    scaler = scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    x_train=torch.from_numpy(x_train.astype(np.float32))
    x_test=torch.from_numpy(x_test.astype(np.float32))
    y_train=torch.from_numpy(y_train.to_numpy().reshape(-1,1).astype(np.float32))
    y_test=torch.from_numpy(y_test.to_numpy().reshape(-1,1).astype(np.float32))

    net = nn.Sequential(nn.Linear(num_features,6),nn.ReLU(),nn.Linear(6,1),nn.Sigmoid())
    
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(net.parameters(),lr=0.3)
    
    for iter in range(1000):
      optimizer.zero_grad()
      outputs = net(x_train)
      loss = criterion(outputs,y_train)
      loss.backward()
      optimizer.step()

    with torch.no_grad():
      y_train_pred = net(x_train).round()
      y_test_pred = net(x_test).round()

    train_accuracies.append( ((y_train_pred == y_train).sum() / len(y_train)).numpy())
    val_accuracies.append(((y_test_pred == y_test).sum() / len(y_test)).numpy())
  
  print('Train accuracy: ',np.mean(train_accuracies))
  print('Validation accuracy: ',np.mean(val_accuracies))

In [398]:
evaluation_kfold(x,y)

Train accuracy:  0.86254996
Validation accuracy:  0.85950005
